# JSON Evaluators Guide

Evaluating JSON outputs can be challenging given the structure and data types involved. This guide aims to assist in understanding and utilizing three evaluators designed for JSON string evaluation. These evaluators ensure the integrity, equality, and minimal difference between JSON strings.

## JsonValidityEvaluator

The `JsonValidityEvaluator` is designed to check the validity of a JSON string prediction.


### Overview:
- **Requires Input?**: No
- **Requires Reference?**: No
- **Evaluation Name**: "json"

In [1]:
from langchain.evaluation import JsonValidityEvaluator

evaluator = JsonValidityEvaluator()

prediction = '{"name": "John", "age": 30, "city": "New York"}'

result = evaluator.evaluate_strings(prediction=prediction)
print(result)

{'score': 1}


In [2]:
prediction = '{"name": "John", "age": 30, "city": "New York",}'
result = evaluator.evaluate_strings(prediction=prediction)
print(result)

{'score': 0, 'reasoning': 'Expecting property name enclosed in double quotes: line 1 column 48 (char 47)'}


## JsonEqualityEvaluator

The `JsonEqualityEvaluator` assesses whether a JSON prediction matches a given reference after both are parsed.

### Overview:
- **Requires Input?**: No
- **Requires Reference?**: Yes
- **Evaluation Name**: "parsed_equality"


In [3]:
from langchain.evaluation import JsonEqualityEvaluator

evaluator = JsonEqualityEvaluator()
result = evaluator.evaluate_strings(prediction='{"a": 1}', reference='{"a": 1}')
print(result)

{'score': True}


In [4]:
result = evaluator.evaluate_strings(prediction='{"a": 1}', reference='{"a": 2}')
print(result)

{'score': False}


The evaluator also by default lets you provide a dictionary directly

In [5]:
result = evaluator.evaluate_strings(prediction={"a": 1}, reference={"a": 2})
print(result)

{'score': False}


## JsonEditDistanceEvaluator

The `JsonEditDistanceEvaluator` computes a normalized Damerau-Levenshtein distance between two "canonicalized" JSON strings.

### Overview:
- **Requires Input?**: No
- **Requires Reference?**: Yes
- **Evaluation Name**: "parsed_equality"
- **Distance Function**: Damerau-Levenshtein (by default)

_Note: Ensure that `rapidfuzz` is installed or provide an alternative `string_distance` function to avoid an ImportError._

In [7]:
from langchain.evaluation import JsonEditDistanceEvaluator

evaluator = JsonEditDistanceEvaluator()

result = evaluator.evaluate_strings(
    prediction='{"a": 1, "b": 2}', reference='{"a": 1, "b": 3}'
)
print(result)

{'score': 0.07692307692307693}


In [8]:
# The values are canonicalized prior to comparison
result = evaluator.evaluate_strings(
    prediction="""
    {
        "b": 3,
        "a":   1
    }""",
    reference='{"a": 1, "b": 3}',
)
print(result)

{'score': 0.0}


In [11]:
# You can also pass in objects directly
result = evaluator.evaluate_strings(prediction={"a": 1}, reference={"a": 2})
print(result)

{'score': 0.14285714285714285}


## Conclusion

These JSON evaluators offer a robust suite of tools for ensuring
the accuracy and validity of model outputs in JSON format. By integrating these into your evaluation pipeline, you can maintain a high standard of data quality for your projects.